<a href="https://colab.research.google.com/github/Otter2022/AITXhackathon/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')
basepath = '/content/drive/MyDrive/AntlerHackathon/Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import numpy as np

In [92]:
#Data imports

shootouts_path = basepath + '/shootouts.csv'
shootouts = pd.read_csv(shootouts_path)

results_path = basepath + '/results.csv'
results = pd.read_csv(results_path)
countries = [
    "Germany", "France", "Spain", "England", "Italy", "Netherlands", "Portugal",
    "Belgium", "Croatia", "Denmark", "Switzerland", "Sweden", "Poland", "Austria",
    "Serbia", "Norway", "Brazil", "Argentina", "Uruguay", "Colombia", "Chile",
    "Ecuador", "Peru", "Mexico", "United States", "Canada", "Costa Rica", "Honduras",
    "Panama", "Nigeria", "Senegal", "Cameroon", "Egypt", "Ghana", "Morocco",
    "Algeria", "Tunisia", "Ivory Coast", "Japan", "South Korea", "Iran", "Australia",
    "Saudi Arabia", "Qatar", "United Arab Emirates", "Iraq", "China", "New Zealand", "Bosnia and Herzegovina"
]
filtered_results = results[
    (results['date'] >= '2000-01-01') &
    (results['home_team'].isin(countries) | results['away_team'].isin(countries))
]
print(results['home_team'].unique())
goalscorers = basepath + '/goalscorers.csv'
goalscorers = pd.read_csv(goalscorers)
groups = {
    'Group A': ['United States', 'Germany', 'Nigeria', 'Japan'],
    'Group B': ['Mexico', 'France', 'Senegal', 'South Korea'],
    'Group C': ['Canada', 'Brazil', 'Egypt', 'Iran'],
    'Group D': ['Argentina', 'England', 'Morocco', 'Australia'],
    'Group E': ['Spain', 'Colombia', 'Cameroon', 'Saudi Arabia'],
    'Group F': ['Netherlands', 'Uruguay', 'Ghana', 'Qatar'],
    'Group G': ['Italy', 'Croatia', 'Algeria', 'United Arab Emirates'],
    'Group H': ['Belgium', 'Portugal', 'Tunisia', 'Iraq'],
    'Group I': ['Denmark', 'Switzerland', 'Ivory Coast', 'China'],
    'Group J': ['Sweden', 'Ecuador', 'Mali', 'Panama'],
    'Group K': ['Poland', 'Peru', 'South Africa', 'New Zealand'],
    'Group L': ['Norway', 'Chile', 'Costa Rica', 'Bosnia and Herzegovina']
}


['Scotland' 'England' 'Wales' 'Northern Ireland' 'United States' 'Uruguay'
 'Austria' 'Hungary' 'Argentina' 'Belgium' 'France' 'Guernsey' 'Jersey'
 'Netherlands' 'Czechoslovakia' 'Alderney' 'Switzerland' 'Sweden'
 'Germany' 'Italy' 'Chile' 'Norway' 'Finland' 'Luxembourg' 'Russia'
 'Denmark' 'Catalonia' 'Philippines' 'Basque Country' 'China PR' 'Brazil'
 'Japan' 'Paraguay' 'Canada' 'Estonia' 'Costa Rica' 'Guatemala' 'Spain'
 'Brittany' 'Poland' 'Yugoslavia' 'New Zealand' 'Romania' 'Latvia'
 'Galicia' 'Portugal' 'Andalusia' 'Australia' 'Lithuania' 'Turkey'
 'Central Spain' 'Mexico' 'Aruba' 'Egypt' 'Republic of Ireland' 'Haiti'
 'Bulgaria' 'Jamaica' 'Kenya' 'Bolivia' 'Peru' 'Honduras' 'Guyana'
 'Uganda' 'Belarus' 'El Salvador' 'Barbados' 'Trinidad and Tobago'
 'Greece' 'Cuba' 'Curaçao' 'Dominica' 'Silesia' 'Guadeloupe' 'Israel'
 'Suriname' 'French Guiana' 'Panama' 'Colombia' 'Venezuela' 'Ecuador'
 'Saint Kitts and Nevis' 'Slovakia' 'Manchukuo' 'Croatia' 'Nicaragua'
 'Afghanistan' 'India' 

In [32]:
def determine_category(row):
    if row['home_score'] > row['away_score']:
        return 2
    elif row['home_score'] == row['away_score']:
        return 1
    else:
        return 0


In [33]:
def determine_final_result(row):
    if row['home_score'] > row['away_score']:
        return 2
    elif row['home_score'] == row['away_score']:
        return 1
    else:
        return 0

In [34]:
results['finals'] = results.apply(determine_category, axis=1)
#Convert the 'date' column to datetime format
results['data'] = pd.to_datetime(results['date'])

#Filter the DataFrame for matches after the year 2000
results['date'] = pd.to_datetime(results['date'], errors='coerce')

results_filtered = results[results['date'].dt.year > 2000]

results_filtered.loc[:, 'finals'] = results_filtered.apply(determine_final_result, axis=1)

# Encode categorical variables
df_encoded = pd.get_dummies(results_filtered[['home_team', 'away_team', 'tournament', 'city', 'country', 'neutral']], drop_first=True)
if 'neutral' in df_encoded:
    df_encoded['neutral'] = df_encoded['neutral'].astype(int)
# Add the 'category' column to the encoded dataframe
df_encoded['finals'] = results_filtered['finals'].values

# Separate features and target
X = df_encoded.drop(columns=['finals'])
y = df_encoded['finals']

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Decision Tree using the ID3 algorithm (use entropy as the criterion)
clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
report = classification_report(y_test, y_pred)
print(report)
single_test_case = pd.DataFrame({
    'home_team': ['France'],
    'away_team': ['Brazil'],
    'tournament': ['Friendly'],
    'city': ['Paris'],
    'country': ['France'],
    'neutral': [False]
})
# Encode the single test case
single_test_case_encoded = pd.get_dummies(single_test_case, drop_first=True)

# Align with training data
single_test_case_encoded = single_test_case_encoded.reindex(columns=X.columns, fill_value=0)

# Predict the outcome for the single test case
single_prediction = clf.predict(single_test_case_encoded)

print("Prediction for the single test case:", single_prediction)


              precision    recall  f1-score   support

           0       0.44      0.49      0.47      1280
           1       0.27      0.25      0.26      1035
           2       0.62      0.60      0.61      2151

    accuracy                           0.49      4466
   macro avg       0.44      0.45      0.44      4466
weighted avg       0.49      0.49      0.49      4466

Prediction for the single test case: [2]


In [38]:
def simulate_match(home_team, away_team, tournament, city, country, neutral):
    match = pd.DataFrame({
        'home_team': [home_team],
        'away_team': [away_team],
        'tournament': [tournament],
        'city': [city],
        'country': [country],
        'neutral': [neutral]
    })
    match_encoded = pd.get_dummies(match, drop_first=True)
    match_encoded = match_encoded.reindex(columns=X.columns, fill_value=0)
    prediction = clf.predict(match_encoded)
    return prediction[0]

# Function to simulate the group stage
def simulate_group_stage(groups):
    group_standings = {group: {team: {'points': 0, 'goals': 0} for team in teams} for group, teams in groups.items()}

    for group, teams in groups.items():
        for i in range(len(teams)):
            for j in range(i + 1, len(teams)):
                home_team = teams[i]
                away_team = teams[j]
                result = simulate_match(home_team, away_team, 'World Cup', 'City', 'Country', False)
                print(f"Match: {home_team} vs {away_team} - Result: {'Home win' if result == 2 else 'Draw' if result == 1 else 'Away win'}")
                if result == 2:
                    group_standings[group][home_team]['points'] += 3
                elif result == 1:
                    group_standings[group][home_team]['points'] += 1
                    group_standings[group][away_team]['points'] += 1
                else:
                    group_standings[group][away_team]['points'] += 3

    return group_standings

# Simulate the group stage
groups = {
    'Group A': ['United States', 'Germany', 'Nigeria', 'Japan'],
    'Group B': ['Mexico', 'France', 'Senegal', 'South Korea'],
    'Group C': ['Canada', 'Brazil', 'Egypt', 'Iran'],
    'Group D': ['Argentina', 'England', 'Morocco', 'Australia'],
    'Group E': ['Spain', 'Colombia', 'Cameroon', 'Saudi Arabia'],
    'Group F': ['Netherlands', 'Uruguay', 'Ghana', 'Qatar'],
    'Group G': ['Italy', 'Croatia', 'Algeria', 'United Arab Emirates'],
    'Group H': ['Belgium', 'Portugal', 'Tunisia', 'Iraq'],
    'Group I': ['Denmark', 'Switzerland', 'Ivory Coast', 'China'],
    'Group J': ['Sweden', 'Ecuador', 'Mali', 'Panama'],
    'Group K': ['Poland', 'Peru', 'South Africa', 'New Zealand'],
    'Group L': ['Norway', 'Chile', 'Costa Rica', 'Bosnia and Herzegovina']
}
group_standings = simulate_group_stage(groups)
print("\nGroup Standings after Group Stage:")
for group, standings in group_standings.items():
    print(group)
    for team, stats in standings.items():
        print(f"{team}: {stats['points']} points")

# Function to simulate knockout stage
def simulate_knockout_stage(group_standings):
    # Get top teams from each group
    knockout_teams = []
    for group, standings in group_standings.items():
        sorted_teams = sorted(standings.items(), key=lambda x: x[1]['points'], reverse=True)
        knockout_teams.append(sorted_teams[0][0])
        knockout_teams.append(sorted_teams[1][0])

    # Get best third-placed teams to fill up to 32 teams
    third_placed_teams = []
    for group, standings in group_standings.items():
        sorted_teams = sorted(standings.items(), key=lambda x: x[1]['points'], reverse=True)
        third_placed_teams.append((sorted_teams[2][0], sorted_teams[2][1]['points']))

    third_placed_teams = sorted(third_placed_teams, key=lambda x: x[1], reverse=True)
    for i in range(8):
        knockout_teams.append(third_placed_teams[i][0])

    # Simulate the knockout rounds
    round_number = 1
    while len(knockout_teams) > 1:
        print(f"\nKnockout Round {round_number}:")
        next_round_teams = []
        for i in range(0, len(knockout_teams), 2):
            if i+1 < len(knockout_teams):
                home_team = knockout_teams[i]
                away_team = knockout_teams[i + 1]
                result = simulate_match(home_team, away_team, 'World Cup', 'City', 'Country', False)
                print(f"Match: {home_team} vs {away_team} - Result: {'Home win' if result == 2 else 'Away win'}")
                if result == 2:
                    next_round_teams.append(home_team)
                else:
                    next_round_teams.append(away_team)
        knockout_teams = next_round_teams
        round_number += 1

    return knockout_teams[0]

# Simulate the knockout stage and determine the winner
world_cup_winner = simulate_knockout_stage(group_standings)
print("\nPredicted World Cup Winner:", world_cup_winner)



Match: United States vs Germany - Result: Home win
Match: United States vs Nigeria - Result: Home win
Match: United States vs Japan - Result: Home win
Match: Germany vs Nigeria - Result: Home win
Match: Germany vs Japan - Result: Home win
Match: Nigeria vs Japan - Result: Home win
Match: Mexico vs France - Result: Home win
Match: Mexico vs Senegal - Result: Home win
Match: Mexico vs South Korea - Result: Home win
Match: France vs Senegal - Result: Home win
Match: France vs South Korea - Result: Home win
Match: Senegal vs South Korea - Result: Home win
Match: Canada vs Brazil - Result: Home win
Match: Canada vs Egypt - Result: Home win
Match: Canada vs Iran - Result: Home win
Match: Brazil vs Egypt - Result: Home win
Match: Brazil vs Iran - Result: Home win
Match: Egypt vs Iran - Result: Home win
Match: Argentina vs England - Result: Home win
Match: Argentina vs Morocco - Result: Home win
Match: Argentina vs Australia - Result: Home win
Match: England vs Morocco - Result: Home win
Match

In [36]:
results['finals'] = results.apply(determine_category, axis=1)

# Convert the 'date' column to datetime format
results['date'] = pd.to_datetime(results['date'], errors='coerce')

# Filter the DataFrame for matches after the year 2000
results_filtered = results[results['date'].dt.year > 2000].copy()

# Encode categorical variables
df_encoded = pd.get_dummies(results_filtered[['home_team', 'away_team', 'tournament', 'city', 'country', 'neutral']], drop_first=True)
if 'neutral' in df_encoded:
    df_encoded['neutral'] = df_encoded['neutral'].astype(int)
# Add the 'finals' column to the encoded dataframe
df_encoded['finals'] = results_filtered['finals'].values

# Separate features and target
X = df_encoded.drop(columns=['finals'])
y = df_encoded['finals']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest
clf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
report = classification_report(y_test, y_pred)
print("Random Forest Classifier report:\n", report)

# Define a single test case
single_test_case = pd.DataFrame({
    'home_team': ['France'],
    'away_team': ['Brazil'],
    'tournament': ['Friendly'],
    'city': ['Paris'],
    'country': ['France'],
    'neutral': [False]
})

# Encode the single test case
single_test_case_encoded = pd.get_dummies(single_test_case, drop_first=True)

# Align with training data
single_test_case_encoded = single_test_case_encoded.reindex(columns=X.columns, fill_value=0)

# Predict the outcome for the single test case
single_prediction = clf.predict(single_test_case_encoded)

print("Prediction for the single test case:", single_prediction)



Random Forest Classifier report:
               precision    recall  f1-score   support

           0       0.80      0.05      0.10      1280
           1       0.00      0.00      0.00      1035
           2       0.49      1.00      0.66      2151

    accuracy                           0.49      4466
   macro avg       0.43      0.35      0.25      4466
weighted avg       0.47      0.49      0.34      4466

Prediction for the single test case: [2]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Train the Random Forest with the resampled data
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
rf_clf.fit(X_resampled, y_resampled)

# Predict on the test set
y_pred_rf = rf_clf.predict(X_test)

# Evaluate the Random Forest model
report_rf = classification_report(y_test, y_pred_rf)
print("Random Forest Classifier report:\n", report_rf)

Random Forest Classifier report:
               precision    recall  f1-score   support

           0       0.45      0.39      0.42      1280
           1       0.28      0.46      0.34      1035
           2       0.61      0.46      0.53      2151

    accuracy                           0.44      4466
   macro avg       0.44      0.44      0.43      4466
weighted avg       0.49      0.44      0.45      4466



In [42]:
scores = results_filtered.loc[:, ['home_team', 'away_team', 'home_score', 'away_score']]

print(scores)

         home_team             away_team  home_score  away_score
25069        Kenya                Zambia           2           1
25070   Costa Rica             Guatemala           5           2
25071        Egypt  United Arab Emirates           2           1
25072     Eswatini                Angola           1           0
25073       Angola               Lesotho           2           0
...            ...                   ...         ...         ...
47394  Netherlands               England           1           2
47395      Uruguay              Colombia           0           1
47396       Canada               Uruguay           2           2
47397        Spain               England           2           1
47398    Argentina              Colombia           1           0

[22330 rows x 4 columns]


In [125]:
# Hypothetical team rankings dictionary
fifa_rankings = {
    "Argentina": 1, "France": 2, "Spain": 3, "England": 4, "Brazil": 5, "Belgium": 6, "Netherlands": 7,
    "Portugal": 8, "Colombia": 9, "Italy": 10, "Uruguay": 11, "Croatia": 12, "Germany": 13, "Morocco": 14,
    "Switzerland": 15, "USA": 16, "Mexico": 17, "Japan": 18, "Senegal": 19, "IR Iran": 20, "Denmark": 21,
    "Austria": 22, "Korea Republic": 23, "Australia": 24, "Ukraine": 25, "Turkey": 26, "Ecuador": 27,
    "Poland": 28, "Sweden": 29, "Wales": 30, "Hungary": 31, "Serbia": 32, "Russia": 33, "Qatar": 34,
    "Panama": 35, "Egypt": 36, "Venezuela": 37, "Ivory Coast": 38, "Nigeria": 39, "Canada": 40,
    "Tunisia": 41, "Peru": 42, "Czech Republic": 43, "Norway": 44, "Scotland": 45, "Bosnia and Herzegovina": 46,
    "Finland": 47, "Slovakia": 48, "Greece": 49, "Turkey": 50, "Romania": 51, "Israel": 52, "Iceland": 53,
    "Slovenia": 54, "North Macedonia": 55, "Bulgaria": 56, "Albania": 57, "Montenegro": 58, "Georgia": 59,
    "Armenia": 60, "Cyprus": 61, "Azerbaijan": 62, "Kosovo": 63, "Kazakhstan": 64, "Lithuania": 65,
    "Belarus": 66, "Luxembourg": 67, "Estonia": 68, "Faroe Islands": 69, "Malta": 70, "Moldova": 71,
    "Liechtenstein": 72, "Gibraltar": 73, "San Marino": 74, "Andorra": 75, "China": 87, "Cameroon": 51,
    "Mali": 52, "South Africa": 57, "Costa Rica": 46, "New Zealand": 122, "Algeria": 30, "United Arab Emirates": 69,
    "Iraq": 55
}

scores_df = results_filtered.loc[:, ['home_score', 'away_score', 'home_team', 'away_team']]
scores_df = scores_df.dropna()

# Initialize dictionaries
# Initialize dictionaries
home_goals = {}
away_goals = {}
home_against = {}
away_against = {}
home_matches = {}
away_matches = {}

# Calculate total goals and match counts
for index, row in scores_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    home_score = row['home_score']
    away_score = row['away_score']

    # Home goals scored
    if home_team in home_goals:
        home_goals[home_team] += home_score
        home_matches[home_team] += 1
    else:
        home_goals[home_team] = home_score
        home_matches[home_team] = 1

    # Away goals scored
    if away_team in away_goals:
        away_goals[away_team] += away_score
        away_matches[away_team] += 1
    else:
        away_goals[away_team] = away_score
        away_matches[away_team] = 1

    # Goals against home team
    if home_team in home_against:
        home_against[home_team] += away_score
    else:
        home_against[home_team] = away_score

    # Goals against away team
    if away_team in away_against:
        away_against[away_team] += home_score
    else:
        away_against[away_team] = home_score

# Calculate averages
avg_home_goals = {team: home_goals[team] / home_matches[team] for team in home_goals}
avg_away_goals = {team: away_goals[team] / away_matches[team] for team in away_goals}
avg_home_against = {team: home_against[team] / home_matches[team] for team in home_against}
avg_away_against = {team: away_against[team] / away_matches[team] for team in away_against}

# Initialize head-to-head dictionaries
head_to_head_home_goals = {}
head_to_head_away_goals = {}

# Populate the head-to-head data from historical match results
for index, row in scores_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    home_score = row['home_score']
    away_score = row['away_score']

    if (home_team, away_team) not in head_to_head_home_goals:
        head_to_head_home_goals[(home_team, away_team)] = []
    if (away_team, home_team) not in head_to_head_away_goals:
        head_to_head_away_goals[(away_team, home_team)] = []

    head_to_head_home_goals[(home_team, away_team)].append(home_score)
    head_to_head_away_goals[(away_team, home_team)].append(away_score)
# Display the averages
print("Average Home Goals Scored:", avg_home_goals)
print("Average Away Goals Scored:", avg_away_goals)
print("Average Goals Against Home Teams:", avg_home_against)
print("Average Goals Against Away Teams:", avg_away_against)


# Function to simulate a match
def simulate_match(home_team, away_team, head_to_head_weight=0.31, fifa_ranking_weight=0.2):
    # Base expected scores
    home_score = avg_home_goals.get(home_team, 1) + avg_away_against.get(away_team, 1)
    away_score = avg_away_goals.get(away_team, 1) + avg_home_against.get(home_team, 1)

    # Head-to-head adjustment
    if (home_team, away_team) in head_to_head_home_goals:
        head_to_head_home_avg = np.mean(head_to_head_home_goals[(home_team, away_team)])
        home_score += head_to_head_weight * head_to_head_home_avg
    if (away_team, home_team) in head_to_head_away_goals:
        head_to_head_away_avg = np.mean(head_to_head_away_goals[(away_team, home_team)])
        away_score += head_to_head_weight * head_to_head_away_avg

    # FIFA ranking adjustment
    if home_team in fifa_rankings and away_team in fifa_rankings:
        home_ranking = fifa_rankings[home_team]
        away_ranking = fifa_rankings[away_team]
        # Lower ranking is better, so invert the values
        if home_ranking < away_ranking:
          if home_ranking - away_ranking > -10:
            home_score += .5
          elif home_ranking - away_ranking > -20:
            home_score += 1
          elif home_ranking - away_ranking > -35:
            home_score += 2
          else:
            home_score += 3

        else:
          if home_ranking - away_ranking > 10:
            away_score += .5
          elif home_ranking - away_ranking > 25:
            away_score += 1
          elif home_ranking - away_ranking > 35:
            away_score += 2
          else:
            away_score += 3

    # Generate random goals using Poisson distribution
    home_goals = np.random.poisson(home_score / 2)
    away_goals = np.random.poisson(away_score / 2)

    return home_goals, away_goals



# Function to simulate the group stage
def simulate_group_stage(groups):
    group_standings = {group: {team: {'points': 0, 'goals_for': 0, 'goals_against': 0, 'goal_difference': 0} for team in teams} for group, teams in groups.items()}

    for group, teams in groups.items():
        for i in range(len(teams)):
            for j in range(i + 1, len(teams)):
                home_team = teams[i]
                away_team = teams[j]
                home_goals, away_goals = simulate_match(home_team, away_team)
                #print(f"Match: {home_team} vs {away_team} - Result: {home_goals}-{away_goals}")

                group_standings[group][home_team]['goals_for'] += home_goals
                group_standings[group][home_team]['goals_against'] += away_goals
                group_standings[group][home_team]['goal_difference'] += home_goals - away_goals

                group_standings[group][away_team]['goals_for'] += away_goals
                group_standings[group][away_team]['goals_against'] += home_goals
                group_standings[group][away_team]['goal_difference'] += away_goals - home_goals

                if home_goals > away_goals:
                    group_standings[group][home_team]['points'] += 3
                elif home_goals < away_goals:
                    group_standings[group][away_team]['points'] += 3
                else:
                    group_standings[group][home_team]['points'] += 1
                    group_standings[group][away_team]['points'] += 1

    return group_standings

# Simulate the group stage
groups = {
    'Group A': ['United States', 'Germany', 'Nigeria', 'Japan'],
    'Group B': ['Mexico', 'France', 'Senegal', 'South Korea'],
    'Group C': ['Canada', 'Brazil', 'Egypt', 'Iran'],
    'Group D': ['Argentina', 'England', 'Morocco', 'Australia'],
    'Group E': ['Spain', 'Colombia', 'Cameroon', 'Saudi Arabia'],
    'Group F': ['Netherlands', 'Uruguay', 'Ghana', 'Qatar'],
    'Group G': ['Italy', 'Croatia', 'Algeria', 'United Arab Emirates'],
    'Group H': ['Belgium', 'Portugal', 'Tunisia', 'Iraq'],
    'Group I': ['Denmark', 'Switzerland', 'Ivory Coast', 'China'],
    'Group J': ['Sweden', 'Ecuador', 'Mali', 'Panama'],
    'Group K': ['Poland', 'Peru', 'South Africa', 'New Zealand'],
    'Group L': ['Norway', 'Chile', 'Costa Rica', 'Bosnia and Herzegovina']
}
total_wins = {}
i = 0
while i < 1000:
    group_standings = simulate_group_stage(groups)
    #print("\nGroup Standings after Group Stage:")
    for group, standings in group_standings.items():
        #print(group)
        for team, stats in standings.items():
            x = 0
            #print(f"{team}: {stats['points']} points, {stats['goals_for']} goals for, {stats['goals_against']} goals against, {stats['goal_difference']} goal difference")

    # Function to simulate knockout stage
    def simulate_knockout_stage(group_standings):
        # Get top teams from each group
        knockout_teams = []
        for group, standings in group_standings.items():
            sorted_teams = sorted(standings.items(), key=lambda x: (x[1]['points'], x[1]['goal_difference'], x[1]['goals_for']), reverse=True)
            knockout_teams.append(sorted_teams[0][0])
            knockout_teams.append(sorted_teams[1][0])

        # Ensure there is an even number of teams for knockout rounds
        # if len(knockout_teams) % 2 != 0:
        #    knockout_teams.append('BYE')  # Add a 'BYE' if necessary

        # Simulate the knockout rounds
        rounds = ["Round of 32", "Round of 16", "Quarterfinals", "Semifinals", "Final"]
        round_number = 0
        while len(knockout_teams) > 1:
            #print(f"\n{rounds[round_number]}:")
            next_round_teams = []
            for i in range(0, len(knockout_teams), 2):
                if i + 1 >= len(knockout_teams):
                    next_round_teams.append(knockout_teams[i])
                    continue
                home_team = knockout_teams[i]
                away_team = knockout_teams[i + 1]
                if home_team == 'BYE':
                    next_round_teams.append(away_team)
                elif away_team == 'BYE':
                    next_round_teams.append(home_team)
                else:
                    home_goals, away_goals = simulate_match(home_team, away_team)
                    #print(f"Match: {home_team} vs {away_team} - Result: {home_goals}-{away_goals}")
                    while home_goals == away_goals:
                        home_goals, away_goals = simulate_match(home_team, away_team)
                    if home_goals > away_goals:
                        next_round_teams.append(home_team)
                    elif away_goals > home_goals:
                        next_round_teams.append(away_team)
                        # In case of a draw, use a simple tie-breaker

            knockout_teams = next_round_teams
            round_number += 1

        #print(f"\nWorld Cup Winner: {knockout_teams[0]}")
        return knockout_teams[0]

    # Simulate the knockout stage
    winner = simulate_knockout_stage(group_standings)
    i += 1  # Increment the loop counter
    if winner in total_wins:
      total_wins[winner] += 1
    else:
      total_wins[winner] = 1

print(total_wins)

Average Home Goals Scored: {'Kenya': 1.348148148148148, 'Costa Rica': 1.6932515337423313, 'Egypt': 2.202127659574468, 'Eswatini': 0.9156626506024096, 'Angola': 1.3622047244094488, 'Togo': 1.4457831325301205, 'Romania': 1.631578947368421, 'Algeria': 1.91156462585034, 'Bosnia and Herzegovina': 1.5688073394495412, 'Qatar': 1.706140350877193, 'Uzbekistan': 1.903225806451613, 'Burkina Faso': 1.6504854368932038, 'India': 1.68503937007874, 'Mauritius': 1.054054054054054, 'Namibia': 1.0888888888888888, 'Nigeria': 1.6888888888888889, 'Tunisia': 1.6987179487179487, 'Uganda': 1.4919354838709677, 'DR Congo': 1.556701030927835, 'Liberia': 1.1612903225806452, 'Serbia': 1.8653846153846154, 'Zimbabwe': 1.489795918367347, 'Chile': 1.751937984496124, 'Bangladesh': 1.054945054945055, 'Hong Kong': 1.93, 'China PR': 1.848314606741573, 'Israel': 1.7096774193548387, 'Kuwait': 1.4858757062146892, 'Iran': 2.165680473372781, 'Ivory Coast': 2.124031007751938, 'Mali': 1.6923076923076923, 'Thailand': 1.78421052631

In [154]:
group_standings = simulate_group_stage(groups)
print("\nGroup Standings after Group Stage:")
for group, standings in group_standings.items():
    #print(group)
    for team, stats in standings.items():
        x = 0
        print(f"{team}: {stats['points']} points, {stats['goals_for']} goals for, {stats['goals_against']} goals against, {stats['goal_difference']} goal difference")

# Function to simulate knockout stage
def simulate_knockout_stage(group_standings):
    # Get top teams from each group
    knockout_teams = []
    for group, standings in group_standings.items():
        sorted_teams = sorted(standings.items(), key=lambda x: (x[1]['points'], x[1]['goal_difference'], x[1]['goals_for']), reverse=True)
        knockout_teams.append(sorted_teams[0][0])
        knockout_teams.append(sorted_teams[1][0])

    # Ensure there is an even number of teams for knockout rounds
    # if len(knockout_teams) % 2 != 0:
    #    knockout_teams.append('BYE')  # Add a 'BYE' if necessary

    # Simulate the knockout rounds
    rounds = ["Round of 32", "Round of 16", "Quarterfinals", "Semifinals", "Final"]
    round_number = 0
    while len(knockout_teams) > 1:
        print(f"\n{rounds[round_number]}:")
        next_round_teams = []
        for i in range(0, len(knockout_teams), 2):
            if i + 1 >= len(knockout_teams):
                next_round_teams.append(knockout_teams[i])
                continue
            home_team = knockout_teams[i]
            away_team = knockout_teams[i + 1]
            if home_team == 'BYE':
                next_round_teams.append(away_team)
            elif away_team == 'BYE':
                next_round_teams.append(home_team)
            else:
                home_goals, away_goals = simulate_match(home_team, away_team)
                while home_goals == away_goals:
                    home_goals, away_goals = simulate_match(home_team, away_team)
                print(f"Match: {home_team} vs {away_team} - Result: {home_goals}-{away_goals}")
                if home_goals > away_goals:
                    next_round_teams.append(home_team)
                elif away_goals > home_goals:
                    next_round_teams.append(away_team)
                    # In case of a draw, use a simple tie-breaker

        knockout_teams = next_round_teams
        round_number += 1

    #print(f"\nWorld Cup Winner: {knockout_teams[0]}")
    return knockout_teams[0]

# Simulate the knockout stage
winner = simulate_knockout_stage(group_standings)
print(winner)



Group Standings after Group Stage:
United States: 9 points, 11 goals for, 4 goals against, 7 goal difference
Germany: 6 points, 8 goals for, 4 goals against, 4 goal difference
Nigeria: 3 points, 3 goals for, 6 goals against, -3 goal difference
Japan: 0 points, 2 goals for, 10 goals against, -8 goal difference
Mexico: 6 points, 6 goals for, 4 goals against, 2 goal difference
France: 7 points, 7 goals for, 3 goals against, 4 goal difference
Senegal: 4 points, 6 goals for, 6 goals against, 0 goal difference
South Korea: 0 points, 2 goals for, 8 goals against, -6 goal difference
Canada: 0 points, 6 goals for, 12 goals against, -6 goal difference
Brazil: 6 points, 11 goals for, 8 goals against, 3 goal difference
Egypt: 3 points, 6 goals for, 9 goals against, -3 goal difference
Iran: 9 points, 10 goals for, 4 goals against, 6 goal difference
Argentina: 6 points, 6 goals for, 4 goals against, 2 goal difference
England: 9 points, 9 goals for, 2 goals against, 7 goal difference
Morocco: 3 poin